In [105]:
import pandas as pd

###  Volume-Slippage Relationship:

#### Factors Influencing Slippage
- Order Size: Larger orders are more likely to move the market price.
- Liquidity: Lower liquidity can lead to higher slippage.
- Volatility: High volatility can cause prices to move quickly, leading to higher slippage.
- Order Type: Market orders tend to have more slippage compared to limit orders.

In this data set since we are given the 'Avg of Daily Equity Traded Val over 3 Months' we have decided to use an empirical model that relate trade volume to slippage.

A common model assumes that slippage increases with the square root of the trade size.


In [106]:

def estimate_slippage(volume, avg_daily_volume, impact_factor=0.5):
    """
    Estimate slippage using a volume-impact model.
    :param volume: Trade volume
    :param avg_daily_volume: Average daily volume
    :param impact_factor: Empirical factor (default is 0.5 for square root relationship)
    :return: Estimated slippage percentage
    """
    return impact_factor * (volume / (avg_daily_volume * 1_000_00_0)) ** 0.5

In [107]:
## Total Trading Cost:
avg_daily_volume = 685.90 # Using TOYOTA MOTOR CORP as example (given raw data is in mn we leave it in mn)
trade_volume = 500_000 # 500k trade

slippage_estimated = estimate_slippage(trade_volume, avg_daily_volume)
print(f"Estimated Slippage: {slippage_estimated:.5f}")

Estimated Slippage: 0.01350


###  Total Trading Cost:

Calculating trading costs accurately is crucial for evaluating and optimizing a quantitative trading strategy. Trading costs typically include explicit costs (like commissions and fees) and implicit costs (like bid-ask spreads and slippage).

#### Components of Trading Costs
- Commissions: Fixed costs per trade or per share.
- Fees: Exchange or regulatory fees.
- Bid-Ask Spread: The difference between the bid (buy) and ask (sell) prices.
- Slippage: The difference between the expected price and the actual price due to market impact.




$$
    Total~Trading~Cost = Commision + Spread~Cost + Slippage
$$

In [108]:
## Parameters
commission_per_trade = 0.01  # 1% commission per trade
bid_ask_spread = 0.002  # 0.2% of the trade price

def calculate_trading_costs(trades, commission_per_trade, bid_ask_spread):
    trade_volume = abs(trades['quantity']) * trades['price']
    
    # Calculate commission cost
    trades['commission'] = commission_per_trade * trade_volume
    
    # Calculate bid-ask spread cost
    trades['bid_ask_spread_cost'] = bid_ask_spread * trade_volume
    
    # Calculate slippage cost
    
    # using parameter
    # slippage_percentage = 0.05
    # trades['slippage_cost'] = slippage_percentage * abs(trades['quantity']) * trades['price']

    # estimate using volume slippage relationship
    trades['slippage_cost'] = estimate_slippage(trade_volume, trades['avg_daily_volume']) * trade_volume
    
    # Total cost per trade
    trades['total_cost'] = trades['commission'] + trades['bid_ask_spread_cost'] + trades['slippage_cost']
    
    return trades

In [109]:
# Example: Calculate trading cost

# Sample trades DataFrame
data = {
    'date': ['2023-06-01', '2023-06-02', '2023-06-03'],
    'symbol': ['AAPL', 'AAPL', 'GOOG'],
    'quantity': [100, -50, 150],  # Positive for buy, negative for sell
    'avg_daily_volume': [67.39, 67.39, 22.8], # in mn
    'price': [150.00, 151.00, 2500.00]
}
trades = pd.DataFrame(data)

trades_with_costs = calculate_trading_costs(trades, commission_per_trade, bid_ask_spread)

print(trades_with_costs)

         date symbol  quantity  avg_daily_volume   price  commission  \
0  2023-06-01   AAPL       100             67.39   150.0       150.0   
1  2023-06-02   AAPL       -50             67.39   151.0        75.5   
2  2023-06-03   GOOG       150             22.80  2500.0      3750.0   

   bid_ask_spread_cost  slippage_cost    total_cost  
0                 30.0     111.894609    291.894609  
1                 15.1      39.956984    130.556984  
2                750.0   24046.367627  28546.367627  
